In [1]:
import selenium
import pandas as pd
import time
from bs4 import BeautifulSoup

from selenium import webdriver

# Importing required Exceptions which needs to handled
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
#Importing requests
import requests

# importing regex
import re
from selenium.webdriver.support.ui import WebDriverWait

# Q.1. 
Write a python program which searches all the product under a particular product from www.amazon.in. 
The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search 
for guitars.

In [2]:
driver=webdriver.Chrome('Chromedriver.exe')
driver.get('https://www.amazon.in')
search=input()
driver.find_element_by_xpath("//input[@id='twotabsearchtextbox']").send_keys(search)
driver.find_element_by_xpath('//span[@id="nav-search-submit-text"]').click()

<ipython-input-2-1725341b9f90>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome('Chromedriver.exe')


guitar


<ipython-input-2-1725341b9f90>:4: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath("//input[@id='twotabsearchtextbox']").send_keys(search)
<ipython-input-2-1725341b9f90>:5: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//span[@id="nav-search-submit-text"]').click()


# 2. 
In the above question, now scrape the following details of each product listed in first 3 pages of your 
search results and save it in a data frame and csv. In case if any product has less than 3 pages in search 
results then scrape all the products available under that product name. Details to be scraped are: "Brand 
Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and 
“Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [3]:
url=[]
brand=[]
name=[]
price=[]
exc=[]
exp_delivery=[]
avail=[]


#Scraping 3 pages
for i in range(3):
    
    #URL
    for j in driver.find_elements_by_xpath('//h2[@class="a-size-mini a-spacing-none a-color-base s-line-clamp-4"]/a'):
        url.append(j.get_attribute('href'))

    #Clicking on Next Button
    try:
        delay=10
        WebDriverWait(driver,delay).until(EC.presence_of_element_located((By.CLASS_NAME,'a-last')))
        driver.find_element_by_xpath("//a[@class='s-pagination-item s-pagination-button']").click()
    except TimeoutException as e:
        break
    time.sleep(3)
        
                
#For every URL
for u in url:
    driver.get(u)
        
    #Brand   
    try:
        table=driver.find_elements_by_xpath('//table[@class="a-normal a-spacing-micro"]/tbody/tr/td')
        found=0
        for index,element in enumerate(table):
            if element.text=='Brand':
                found=1
                brand.append(table[index+1].text)
                break
        if found==0:
            brand.append('-')
    except NoSuchElementException as e:
        try:
            table1=driver.find_elements_by_xpath('//table[@id="productDetails_techSpec_section_1"]/tbody/tr')
            found=0
            for i in table1:
                if i.text.split()[0]=='Brand':
                    found=1
                    l=i.text.split()[1:]
                    s=''
                    for words in l:
                        s=s+' '+words
                    brand.append(s)
                    break
            if found==0:
                brand.append('-')
        except NoSuchElementException as e:
            brand.append('-')
    
    #Name
    name.append(driver.find_element_by_xpath('//span[@id="productTitle"]').text)
            
    #price
    try:
        price.append(driver.find_element_by_xpath('//span[@class="a-price"]').text)
    except NoSuchElementException as e:
        try:
            price.append(driver.find_element_by_xpath('//span[@class="a-price"]').text)
        except NoSuchElementException as e:
            try:
                price.append(driver.find_element_by_xpath('//span[@class="a-price"]').text)
            except NoSuchElementException as e:
                price.append('-')
            
    #Return/Exchange
    try:
        exc.append(driver.find_element_by_xpath('//div[@data-name="RETURNS_POLICY"]/span').text)
    except NoSuchElementException as e:
        exc.append('-')
        
    #Expected Delivery
    try:
        exp_delivery.append(driver.find_element_by_xpath('//div[@id="deliveryBlockMessage"]/b').text)
    except NoSuchElementException as e:
        try:
            if driver.find_element_by_xpath('//*[@id="outOfStock"]')!='':
                exp_delivery.append('Out of Stock')
        except NoSuchElementException as e:
            exp_delivery.append('-')
        
    #Availibility
    try:
        avail.append(driver.find_element_by_xpath('//div[@id="availability"]/span').text)
    except NoSuchElementException as e:
        avail.append('Out of Stock')
        
    
data=pd.DataFrame()
data['Brand']=brand
data['Name']=name
data['Price']=price
data['Return/Exchange']=exc
data['Expected Delivery']=exp_delivery
data['Availability']=avail
data['URL']=url

<ipython-input-3-f0507410cb47>:14: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  for j in driver.find_elements_by_xpath('//h2[@class="a-size-mini a-spacing-none a-color-base s-line-clamp-4"]/a'):
<ipython-input-3-f0507410cb47>:21: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath("//a[@class='s-pagination-item s-pagination-button']").click()
<ipython-input-3-f0507410cb47>:33: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  table=driver.find_elements_by_xpath('//table[@class="a-normal a-spacing-micro"]/tbody/tr/td')
<ipython-input-3-f0507410cb47>:61: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  name.append(driver.find_element_by_xpath('//span[@id="productTitle"]').text)
<ipython-i

In [4]:
data

,Brand,Name,Price,Return/Exchange,Expected Delivery,Availability,URL
0,Kadence,Kadence Frontier Jumbo Semi Acoustic Guitar Wi...,,7 Days Replacement,-,In stock.,https://www.amazon.in/gp/slredirect/picassoRed...
1,VAULT,Vault Traveller 34 Inch Acoustic Guitar With B...,,7 Days Replacement,-,In stock.,https://www.amazon.in/gp/slredirect/picassoRed...
2,KETOSTICS,Ketostics™ Venus Special Cutaway Rosewood 41 I...,,7 Days Replacement,-,In stock.,https://www.amazon.in/gp/slredirect/picassoRed...
3,Guitar Bro,GUITAR BRO - COMBO (Black Acoustic Guitar for ...,,7 Days Replacement,-,In stock.,https://www.amazon.in/gp/slredirect/picassoRed...
4,JUAREZ,"Juarez Acoustic Guitar, 38 Inch Curved Body Cu...",,7 Days Replacement,-,In stock.,https://www.amazon.in/Juarez-Acoustic-Guitar-C...
...,...,...,...,...,...,...,...
183,YAMAHA,Yamaha FX280 Semi acoustic Dreadnought Guitar ...,,7 Days Replacement,-,Only 2 left in stock.,https://www.amazon.in/Yamaha-FX280-acoustic-Dr...
184,KETOSTICS,Ketostics® Givson Venus Special Guitar (RED) V...,,7 Days Replacement,-,In stock.,https://www.amazon.in/gp/slredirect/picassoRed...
185,Kadence,"Kadence Guitar Acoustica Series, 41” Jumbo Siz...",,7 Days Replacement,-,In stock.,https://www.amazon.in/gp/slredirect/picassoRed...
186,KETOSTICS,Givson Venus Special Cutaway Rosewood 41 Inch ...,,7 Days Replacement,-,In stock.,https://www.amazon.in/gp/slredirect/picassoRed...


In [5]:
data.dtypes

Brand                object
Name                 object
Price                object
Return/Exchange      object
Expected Delivery    object
Availability         object
URL                  object
dtype: object

# 3.
Write a python program to access the search bar and search button on images.google.com and scrape 10
images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.

In [6]:
def images(picture):
    driver=webdriver.Chrome('Chromedriver.exe')
    driver.get('https://images.google.com')

    box=driver.find_element_by_xpath('//input[@class="gLFyf gsfi"]')
    box.send_keys(picture)
    box.send_keys(Keys.ENTER)

    for i in range(5):
        driver.execute_script("window.scrollBy(0,10000)")
        time.sleep(2)
        try:
            driver.find_element_by_xpath('//input[@value="Show more results"]').click()
        except ElementNotInteractableException:
            pass

    image_count=0
    for img in driver.find_elements_by_xpath('//img[@class="rg_i Q4LuWd"]'):
        img.screenshot('C:\\Users\\aush1\\OneDrive\\Documents\\Datatrained\\Internship\\Project4 Selenium\\images\\'+picture+str(image_count)+'.png')
        image_count+=1
        if image_count==100:
            break

In [7]:
images('fruits')

<ipython-input-6-36cb164f04a7>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome('Chromedriver.exe')
<ipython-input-6-36cb164f04a7>:5: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  box=driver.find_element_by_xpath('//input[@class="gLFyf gsfi"]')
<ipython-input-6-36cb164f04a7>:13: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//input[@value="Show more results"]').click()
<ipython-input-6-36cb164f04a7>:18: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  for img in driver.find_elements_by_xpath('//img[@class="rg_i Q4LuWd"]'):


In [8]:
images('cars')

<ipython-input-6-36cb164f04a7>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome('Chromedriver.exe')
<ipython-input-6-36cb164f04a7>:5: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  box=driver.find_element_by_xpath('//input[@class="gLFyf gsfi"]')
<ipython-input-6-36cb164f04a7>:13: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//input[@value="Show more results"]').click()
<ipython-input-6-36cb164f04a7>:18: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  for img in driver.find_elements_by_xpath('//img[@class="rg_i Q4LuWd"]'):


In [9]:
images('cakes')

<ipython-input-6-36cb164f04a7>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome('Chromedriver.exe')
<ipython-input-6-36cb164f04a7>:5: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  box=driver.find_element_by_xpath('//input[@class="gLFyf gsfi"]')
<ipython-input-6-36cb164f04a7>:13: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//input[@value="Show more results"]').click()
<ipython-input-6-36cb164f04a7>:18: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  for img in driver.find_elements_by_xpath('//img[@class="rg_i Q4LuWd"]'):


# 4.
Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on
www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be 
scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, 
“Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the 
details is missing then replace it by “- “. Save your results in a dataframe and CSV.


In [12]:
def mobileInfo(phone_name,url):
    driver=webdriver.Chrome('Chromedriver.exe')
    driver.get(url)

    #Closing the login prompt
    try:
        driver.find_element_by_xpath('//button[@class="_2KpZ6l _2doB4z"]').click()
    except NoSuchElementException:
        pass

    #Typing phone name and clicking search button
    driver.find_element_by_xpath('//div[@class="_3OO5Xc"]/input').send_keys(phone_name)
    driver.find_element_by_xpath('//button[@class="L0Z3Pu"]').click()

    #Initializing list for all the information to be extracted
    url=[]
    brand=[]
    name=[]
    colour=[]
    ram=[]
    storage=[]
    pcamera=[]
    scamera=[]
    dispsize=[]
    dispreso=[]
    processor=[]
    processor_cores=[]
    batt_capacity=[]
    price=[]

    #Extracting URL of all the phones
    delay=10
    WebDriverWait(driver,delay).until(EC.presence_of_element_located((By.CLASS_NAME,'_1fQZEK')))
    for i in driver.find_elements_by_xpath('//a[@class="_1fQZEK"]'):
        url.append(i.get_attribute('href'))

    #Iterating through the URL list and extracting all phone's info
    for i in url:
        driver.get(i)
        time.sleep(2)

        #Flag variables
        n=0
        c=0
        r=0
        s=0
        pca=0
        sc=0
        ds=0
        dr=0
        pt=0
        pc=0
        bc=0
        
        #Extracting phone Brand
        brand.append(driver.find_element_by_xpath('//h1[@class="yhB1nd"]/span').text.split()[0])

        #Extracting phone Price
        price.append(driver.find_element_by_xpath('//div[@class="_30jeq3 _16Jk6d"]').text)

        #Clicking on the readmore label
        driver.find_element_by_xpath('//button[@class="_2KpZ6l _1FH0tX"]').click()
        time.sleep(2)

        #Extracting all the specification's table rows
        specs=driver.find_elements_by_xpath('//tr[@class="_1s_Smc row"]')

        for row in specs:
            label=row.find_element_by_xpath('td[1]').text

            #Extracting phone Name
            if label=='Model Name':
                name.append(row.find_element_by_xpath('td[2]').text)
                n=1

            #Extracting phone Color
            if label=='Color':
                colour.append(row.find_element_by_xpath('td[2]').text)
                c=1

            ##Extracting phone RAM
            if label=='RAM':
                ram.append(row.find_element_by_xpath('td[2]').text)
                r=1

            #Extracting phone Storage
            if label=='Internal Storage':
                storage.append(row.find_element_by_xpath('td[2]').text)
                s=1

            #Extracting phone Primary Camera
            if label=='Primary Camera':
                pcamera.append(row.find_element_by_xpath('td[2]').text)
                pca=1

            #Extracting phone Secondary Camera
            if label=='Secondary Camera':
                scamera.append(row.find_element_by_xpath('td[2]').text)
                sc=1

            #Extracting phone Display Size
            if label=='Display Size':
                dispsize.append(row.find_element_by_xpath('td[2]').text)
                ds=1

            #Extracting phone Resolution
            if label=='Resolution':
                dispreso.append(row.find_element_by_xpath('td[2]').text)
                dr=1

            #Extracting phone Processor
            if label=='Processor Type':
                processor.append(row.find_element_by_xpath('td[2]').text)
                pt=1

            #Extracting phone Processor Core
            if label=='Processor Core':
                processor_cores.append(row.find_element_by_xpath('td[2]').text)
                pc=1

            #Extracting phone Battery Capacity
            if label=='Battery Capacity':
                batt_capacity.append(row.find_element_by_xpath('td[2]').text)
                bc=1

        #Checking Flag variables
        if n==0:
            name.append('-')
        if c==0:
            colour.append('-')
        if r==0:
            ram.append('-')
        if s==0:
            storage.append('-')
        if pca==0:
            pcamera.append('-')
        if sc==0:
            scamera.append('-')
        if ds==0:
            dispsize.append('-')
        if dr==0:
            dispreso.append('-')
        if pt==0:
            processor.append('-')
        if pc==0:
            processor_cores.append('-')
        if bc==0:
            batt_capacity.append('-')

    #Creating DataFrame
    data=pd.DataFrame()
    data['Name']=name
    data['Brand']=brand
    data['Colour']=colour
    data['Ram']=ram
    data['Storage']=storage
    data['Primary Camera']=pcamera
    data['Secondary Camera']=scamera
    data['Display Size']=dispsize
    data['Display Resolution']=dispreso
    data['Processor']=processor
    data['Processor Cores']=processor_cores
    data['Battery Capacity']=batt_capacity
    data['Price']=price
    data['URL']=url
    return data

In [13]:
mobile='samsung S20'
df=mobileInfo(mobile,'https://www.flipkart.com/')
df

<ipython-input-12-c891283b069b>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome('Chromedriver.exe')
<ipython-input-12-c891283b069b>:7: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//button[@class="_2KpZ6l _2doB4z"]').click()
<ipython-input-12-c891283b069b>:12: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//div[@class="_3OO5Xc"]/input').send_keys(phone_name)
<ipython-input-12-c891283b069b>:13: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//button[@class="L0Z3Pu"]').click()
<ipython-input-12-c891283b069b>:34: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) in

,Name,Brand,Colour,Ram,Storage,Primary Camera,Secondary Camera,Display Size,Display Resolution,Processor,Processor Cores,Battery Capacity,Price,URL
0,Galaxy S20,SAMSUNG,Cloud Blue,8 GB,128 GB,64MP + 12MP + 12MP,10MP Front Camera,15.75 cm (6.2 inch),3200 x 1440 Pixels,Exynos 990,Octa Core,4000 mAh,"₹49,980",https://www.flipkart.com/samsung-galaxy-s20-cl...
1,Galaxy S20,SAMSUNG,Cloud Pink,8 GB,128 GB,64MP + 12MP + 12MP,10MP Front Camera,15.75 cm (6.2 inch),3200 x 1440 Pixels,Exynos 990,Octa Core,4000 mAh,"₹70,499",https://www.flipkart.com/samsung-galaxy-s20-cl...
2,Galaxy S20,SAMSUNG,Cosmic Gray,8 GB,128 GB,64MP + 12MP + 12MP,10MP Front Camera,15.75 cm (6.2 inch),3200 x 1440 Pixels,Exynos 990,Octa Core,4000 mAh,"₹52,905",https://www.flipkart.com/samsung-galaxy-s20-co...
3,S20 FE 5G,SAMSUNG,Cloud Mint,8 GB,128 GB,12MP Rear Camera,-,16.51 cm (6.5 inch),1080 x 2400$$pixel,-,Octa Core,4500 mAh,"₹36,262",https://www.flipkart.com/samsung-s20-fe-5g-clo...
4,Galaxy A20s,SAMSUNG,Black,4 GB,64 GB,13MP + 8MP + 5MP,8MP Front Camera,16.51 cm (6.5 inch),720 x 1520 Pixels,Qualcomm SDM450-B01,Octa Core,4000 mAh,"₹12,990",https://www.flipkart.com/samsung-galaxy-a20s-b...
5,S20 FE 5G,SAMSUNG,Cloud Navy,8 GB,128 GB,12MP Rear Camera,-,16.51 cm (6.5 inch),2400 x 1080$$pixel,-,Octa Core,4500 mAh,"₹35,898",https://www.flipkart.com/samsung-s20-fe-5g-clo...
6,Galaxy S20 FE,SAMSUNG,Cloud Navy,8 GB,128 GB,12MP + 12MP + 8MP Rear Camera,32MP Front Camera,16.51 cm (6.5 inch),2400 x 1080 Pixels,Exynos Octa Core,Octa Core,4500 mAh,"₹48,440",https://www.flipkart.com/samsung-galaxy-s20-fe...
7,Galaxy A20s,SAMSUNG,Green,3 GB,32 GB,13MP + 8MP + 5MP,8MP Front Camera,16.51 cm (6.5 inch),720 x 1520 Pixels,Qualcomm SDM450-B01,Octa Core,4000 mAh,"₹10,775",https://www.flipkart.com/samsung-galaxy-a20s-g...
8,Galaxy A20s,SAMSUNG,Blue,3 GB,32 GB,13MP + 8MP + 5MP,8MP Front Camera,16.51 cm (6.5 inch),720 x 1520 Pixels,Qualcomm SDM450-B01,Octa Core,4000 mAh,"₹11,350",https://www.flipkart.com/samsung-galaxy-a20s-b...
9,Galaxy S20 Ultra,SAMSUNG,Cosmic Gray,12 GB,128 GB,108 MP + 48 MP + 12 MP + VGA Depth Camera,40MP Front Camera,17.53 cm (6.9 inch),3200 x 1440 Pixels,Exynos 990,Octa Core,5000 mAh,"₹97,999",https://www.flipkart.com/samsung-galaxy-s20-ul...


In [14]:
data= pd.DataFrame(df)
data

,Name,Brand,Colour,Ram,Storage,Primary Camera,Secondary Camera,Display Size,Display Resolution,Processor,Processor Cores,Battery Capacity,Price,URL
0,Galaxy S20,SAMSUNG,Cloud Blue,8 GB,128 GB,64MP + 12MP + 12MP,10MP Front Camera,15.75 cm (6.2 inch),3200 x 1440 Pixels,Exynos 990,Octa Core,4000 mAh,"₹49,980",https://www.flipkart.com/samsung-galaxy-s20-cl...
1,Galaxy S20,SAMSUNG,Cloud Pink,8 GB,128 GB,64MP + 12MP + 12MP,10MP Front Camera,15.75 cm (6.2 inch),3200 x 1440 Pixels,Exynos 990,Octa Core,4000 mAh,"₹70,499",https://www.flipkart.com/samsung-galaxy-s20-cl...
2,Galaxy S20,SAMSUNG,Cosmic Gray,8 GB,128 GB,64MP + 12MP + 12MP,10MP Front Camera,15.75 cm (6.2 inch),3200 x 1440 Pixels,Exynos 990,Octa Core,4000 mAh,"₹52,905",https://www.flipkart.com/samsung-galaxy-s20-co...
3,S20 FE 5G,SAMSUNG,Cloud Mint,8 GB,128 GB,12MP Rear Camera,-,16.51 cm (6.5 inch),1080 x 2400$$pixel,-,Octa Core,4500 mAh,"₹36,262",https://www.flipkart.com/samsung-s20-fe-5g-clo...
4,Galaxy A20s,SAMSUNG,Black,4 GB,64 GB,13MP + 8MP + 5MP,8MP Front Camera,16.51 cm (6.5 inch),720 x 1520 Pixels,Qualcomm SDM450-B01,Octa Core,4000 mAh,"₹12,990",https://www.flipkart.com/samsung-galaxy-a20s-b...
5,S20 FE 5G,SAMSUNG,Cloud Navy,8 GB,128 GB,12MP Rear Camera,-,16.51 cm (6.5 inch),2400 x 1080$$pixel,-,Octa Core,4500 mAh,"₹35,898",https://www.flipkart.com/samsung-s20-fe-5g-clo...
6,Galaxy S20 FE,SAMSUNG,Cloud Navy,8 GB,128 GB,12MP + 12MP + 8MP Rear Camera,32MP Front Camera,16.51 cm (6.5 inch),2400 x 1080 Pixels,Exynos Octa Core,Octa Core,4500 mAh,"₹48,440",https://www.flipkart.com/samsung-galaxy-s20-fe...
7,Galaxy A20s,SAMSUNG,Green,3 GB,32 GB,13MP + 8MP + 5MP,8MP Front Camera,16.51 cm (6.5 inch),720 x 1520 Pixels,Qualcomm SDM450-B01,Octa Core,4000 mAh,"₹10,775",https://www.flipkart.com/samsung-galaxy-a20s-g...
8,Galaxy A20s,SAMSUNG,Blue,3 GB,32 GB,13MP + 8MP + 5MP,8MP Front Camera,16.51 cm (6.5 inch),720 x 1520 Pixels,Qualcomm SDM450-B01,Octa Core,4000 mAh,"₹11,350",https://www.flipkart.com/samsung-galaxy-a20s-b...
9,Galaxy S20 Ultra,SAMSUNG,Cosmic Gray,12 GB,128 GB,108 MP + 48 MP + 12 MP + VGA Depth Camera,40MP Front Camera,17.53 cm (6.9 inch),3200 x 1440 Pixels,Exynos 990,Octa Core,5000 mAh,"₹97,999",https://www.flipkart.com/samsung-galaxy-s20-ul...


# 5. 
Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google
maps.


In [15]:
def coordinates(place):
    driver=webdriver.Chrome('Chromedriver.exe')
    driver.get('https://www.google.co.in/maps')
    
    search=driver.find_element_by_xpath('//input[@id="searchboxinput"]')
    search.send_keys(place) #typing place name on the searchbox
    search.send_keys(Keys.ENTER) #pressing enter in the search box
    time.sleep(3)
    
    #Getting the url of the page
    url_string = driver.current_url
    
    #Extracting the latitudes and longitudes from the url
    lat_long = re.findall('@(.*),',url_string)
    lat_long=lat_long[0].split(',')
    print('Latitude : {}'.format(lat_long[0]))
    print('Longitude : {}'.format(lat_long[1]))

In [16]:
coordinates('pune')

<ipython-input-15-59e4c11d924e>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome('Chromedriver.exe')
<ipython-input-15-59e4c11d924e>:5: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  search=driver.find_element_by_xpath('//input[@id="searchboxinput"]')


Latitude : 18.5245649
Longitude : 73.7228795


# 6.
Write a program to scrap details of all the funding deals for second quarter (i.e Jan 21 – March 21) 
from trak.in.

In [30]:
# Activating the chrome browser
driver=webdriver.Chrome("chromedriver.exe") 
time.sleep(2)

# opening trak.in
url = "https://trak.in/"
driver.get(url)
time.sleep(2)

# do click on funding deals
button = driver.find_element_by_xpath('//li[@id="menu-item-51510"]/a').get_attribute('href')
driver.get(button)

# Empty lists
data = {}
data['Date'] = []
data['Startup Name'] = []
data['Industry/Vertical'] = []
data['Sub-Vertical'] = []
data['Location'] = []
data['Investor'] = []
data['Investment Type'] = []
data['Amount(in USD)'] = []


for i in range(54,57):
    # Date
    date = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[2]'.format(i))
    for d in date:
        data['Date'].append(d.text)

    # Startup Name
    startup = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[3]'.format(i))
    for n in startup:
        data['Startup Name'].append(n.text)
    
    # Industry/Vertical
    industry = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[4]'.format(i))
    for n in industry:
        data['Industry/Vertical'].append(n.text)
    
    # Sub-Vertical
    sv = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[5]'.format(i))
    for s in sv:
        data['Sub-Vertical'].append(s.text)

    # Location
    loc = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[6]'.format(i))
    for l in loc:
        data['Location'].append(l.text)
    
    # Investor
    investor = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[7]'.format(i))
    for n in investor:
        data['Investor'].append(n.text)
    
    # Investment Type
    invt = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[8]'.format(i))
    for n in invt:
        data['Investment Type'].append(n.text)
    
    # Amount
    amount = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[9]'.format(i))
    for a in amount:
        data['Amount(in USD)'].append(a.text)
    
df = pd.DataFrame(data)
df

<ipython-input-30-5662415f5944>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome("chromedriver.exe")
<ipython-input-30-5662415f5944>:11: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  button = driver.find_element_by_xpath('//li[@id="menu-item-51510"]/a').get_attribute('href')
<ipython-input-30-5662415f5944>:28: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  dt = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[2]'.format(i))
<ipython-input-30-5662415f5944>:33: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  sn = driver.find_elements_by_xpath('//table[@id="tablepress-{}"]/tbody/tr/td[3]'.format(i))
<ipython-input-30-5662415f5944>:38: DeprecationWarning: find_elements_by_xpath is deprecat

,Date,Startup Name,Industry/Vertical,Sub-Vertical,Location,Investor,Investment Type,Amount(in USD)
0,15/01/2021,Digit Insurance,Financial Services,Insurance Services,Bengaluru,"A91 Partners, Faering Capital, TVS Capital Funds",Venture,"1,80,00,000"
1,28/01/2021,Bombay Shaving Company,Consumer Goods Company,"Shave care, beard care, and skincare products",New Delhi,Reckitt Benckiser,Venture,"6,172,258.50"
2,19/01/2021,DeHaat,AgriTech Startup,online marketplace for farm products and services,Patna,Prosus Ventures,Series C,"30,000,000"
3,19/01/2021,Darwinbox,SaaS,HR Tech,Mumbai,Salesforce Ventures,Seed,"15,000,000"
4,18/01/2021,mfine,Health Tech Startup,AI-powered telemedicine mobile app,Bengaluru,Heritas Capital Management,Venture Round,"16,000,000"
5,18/01/2021,Udayy,EdTech,Online learning platform for kids in class 1-5,Gurgaon,Sequoia Capital,Seed Funding,"15,000,000"
6,11/01/2021,True Elements,Food Startup,Whole Food plant based Nashta,Pune,SIDBI Venture Capital,Series,"100,000,000"
7,13/01/2021,Saveo,B2B E-commerce,Pharmacies,Bengaluru,"Matrix Partners India, RTP Global, others",Seed,"4,000,000"
8,11/02/2021,Doubtnut,Edu Tech,E-Learning Platform,Gurgaon,"SIG Global, Sequoia Capital, WaterBridge Ventu...",Series B,"2,500,000"
9,22/02/2021,Zomato,Hospitality,Online Food Delivery Platform,Gurgaon,"Tiger Global, Kora",Venture,"250,000,000"


# 7. 
Write a program to scrap all the available details of best gaming laptops from digit.in

In [44]:
def bestlaptops():
    driver=webdriver.Chrome('Chromedriver.exe')
    driver.get('https://www.digit.in/') #loading url
    driver.maximize_window() #maximizing window
    
#Clicking on the laptop menu
    driver.find_element_by_xpath('//div[@class="menu"]/ul/li[3]/a').click()
    
#CLicking on best laptops in india
    driver.find_element_by_xpath('//a[text()="Best Gaming Laptops in India"]').click()
    
#Extracting url of each laptop
    url=[]
    for i in driver.find_elements_by_xpath('//span[@class="datahreflink spec"]'):
        url.append(i.get_attribute('data-href'))

#Initializing list to save all the features
    model=[]
    os=[]
    type=[]
    resolution=[]
    dispsize=[]
    wc=[]
    ram=[]
    Ram_type=[]            
    weight=[]
    dimension=[]
    processor=[]
    cs=[]
    cores=[]
    gprocessor=[]
    cache=[]
    sd_type=[]
    sd_capacity=[]
    battery=[]
    battery_type=[]
    power=[]

#Iterating through each url
    for u in url:
        spec=[]
        driver.get(u)
        
#Flag variables
        m=0
        o=0
        lt=0
        reso=0
        d=0
        w=0
        r=0
        rt=0
        lw=0
        ld=0
        p=0
        c=0
        co=0
        gp=0
        ca=0
        sdt=0
        sdc=0
        bb=0
        bt=0
        ps=0
        
        #Iterating each row in the specs table
        for row in driver.find_elements_by_xpath('//div[@class="remaining-details"]/table/tbody/tr'):
            label=row.find_element_by_xpath('td[1]').text

            #Extracting each feature if label matches and setting flag=1 if found
            if label=='Model Name':
                model.append(row.find_element_by_xpath('td[3]').text)
                m=1
            if label=='Operating System (With Version)':
                os.append(row.find_element_by_xpath('td[3]').text)
                o=1
            if label=='Laptop Type':
                type.append(row.find_element_by_xpath('td[3]').text)
                lt=1
            if label=='Resolution':
                resolution.append(row.find_element_by_xpath('td[3]').text)
                reso=1
            if label=='Display Size (In Inches)':
                dispsize.append(row.find_element_by_xpath('td[3]').text)
                d=1
            if label=='Wireless Connectivity':
                wc.append(row.find_element_by_xpath('td[3]').text)
                w=1
            if label=='Ram Included (In Gb)':
                ram.append(row.find_element_by_xpath('td[3]').text)
                r=1
            if label=='Ram Type':
                Ram_type.append(row.find_element_by_xpath('td[2]').text)
                rt=1
            if label=='Laptop Weight (In Kgs)':
                weight.append(row.find_element_by_xpath('td[3]').text)
                lw=1
            if label=='Laptop Dimension (In Mm)':
                dimension.append(row.find_element_by_xpath('td[3]').text)
                ld=1
            if label=='Processor Model Name':
                processor.append(row.find_element_by_xpath('td[3]').text)
                p=1
            if label=='Clock Speed':
                cs.append(row.find_element_by_xpath('td[3]').text)
                c=1
            if label=='Cores':
                cores.append(row.find_element_by_xpath('td[3]').text)
                co=1
            if label=='Graphics Processor':
                gprocessor.append(row.find_element_by_xpath('td[3]').text)
                gp=1
            if label=='Cache L3':
                cache.append(row.find_element_by_xpath('td[3]').text)
                ca=1
            if label=='Storage Drive Type':
                sd_type.append(row.find_element_by_xpath('td[3]').text)
                sdt=1
            if label=='Storage Drive Capacity':
                sd_capacity.append(row.find_element_by_xpath('td[3]').text)
                sdc=1
            if label=='Battery Backup (In Hours)':
                battery.append(row.find_element_by_xpath('td[3]').text)
                bb=1
            if label=='Battery Type':
                battery_type.append(row.find_element_by_xpath('td[3]').text)
                bt=1
            if label=='Power Supply':
                power.append(row.find_element_by_xpath('td[3]').text)
                ps=1
        
        #Appending '-' if flag of a feature is not set
        if m==0:
            model.append('-')
        if o==0:
            os.append('-')
        if lt==0:
            type.append('-')
        if reso==0:
            resolution.append('-')
        if d==0:
            dispsize.append('-')
        if w==0:
            wc.append('-')
        if r==0:
            ram.append('-')
        if rt==0:
            Ram_type.append('-')
        if lw==0:
            weight.append('-')
        if ld==0:
            dimension.append('-')
        if p==0:
            processor.append('-')
        if c==0:
            cs.append('-')
        if co==0:
            cores.append('-')
        if gp==0:
            gprocessor.append('-')
        if ca==0:
            cache.append('-')
        if sdt==0:
            sd_type.append('-')
        if sdc==0:
            sd_capacity.append('-')
        if bb==0:
            battery.append('-')
        if bt==0:
            battery_type.append('-')
        if ps==0:
            power.append('-')

    data=pd.DataFrame()
    data['Model Name']=model
    data['Laptop Type']=type
    data['Operating System']=os
    data['Resolution']=resolution
    data['Display Size']=dispsize
    data['Wireless Connectivity']=wc
    data['Ram Included']=ram
    data['Ram Type']=Ram_type
    data['Laptop Weight (In Kgs)']=weight
    data['Laptop Dimension (In Mm)']=dimension
    data['Processor Model Name']=processor
    data['Clock Speed']=cs
    data['Cores']=cores
    data['Graphics Processor']=gprocessor
    data['Cache L3']=cache
    data['Storage Drive Type']=sd_type
    data['Storage Drive Capacity']=sd_capacity
    data['Battery Backup (In Hours)']=battery
    data['Battery Backup (In Hours)']=battery_type
    data['Power Supply']=power
    return data

In [45]:
df=bestlaptops()
df

<ipython-input-44-b3c05c1f2410>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome('Chromedriver.exe')
<ipython-input-44-b3c05c1f2410>:7: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//div[@class="menu"]/ul/li[3]/a').click()
<ipython-input-44-b3c05c1f2410>:10: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//a[text()="Best Gaming Laptops in India"]').click()
<ipython-input-44-b3c05c1f2410>:14: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=xpath) instead
  for i in driver.find_elements_by_xpath('//span[@class="datahreflink spec"]'):
<ipython-input-44-b3c05c1f2410>:67: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value

,Model Name,Laptop Type,Operating System,Resolution,Display Size,Wireless Connectivity,Ram Included,Ram Type,Laptop Weight (In Kgs),Laptop Dimension (In Mm),Processor Model Name,Clock Speed,Cores,Graphics Processor,Cache L3,Storage Drive Type,Storage Drive Capacity,Battery Backup (In Hours),Power Supply
0,Victus 16 (AMD),-,Windows 10,1920 x 1200,16.1,-,16 GB,:,2.48,37 x 26.2 x 2.4,5th Gen AMD Ryzen 7 5800H,3.2GHz,8,NVIDIA GeForce RTX 3060,16MB,SSD,512 GB,-,-
1,Omen 15,-,Windows 10 Home,1920 x 1080,15.6,-,16 GB,:,2.37,357.9 x 239.7 x 22.5,AMD Ryzen 7 Octa Core-5800H,4.4 GHz,8,NVIDIA GeForce RTX 3060,16 MB,SSD,1 TB,-,-
2,NITRO 5 Ryzen 9,-,Windows 10,1920 x 1080,15.6,-,16 GB,:,2.4,363.4 x 255 x 23.9,AMD Ryzen 9 Octa Core,2.4 GHz,8,NVIDIA GeForce RTX 3070,-,HDD,1 TB,4-cell Li-ion,180 W
3,Stealth 15M,-,Windows 10,1920 x 1080,15.6,-,16 GB,:,1.7,358.3 x 248 x 16.15,Intel Core i7 11th Gen - 11375H,-,-,NVIDIA GeForce RTX 3060,-,SSD,1 TB,3-cell Li-ion,150W
4,ROG Strix Scar 15,-,Windows 10,2560 x 1440,15.6,-,32 GB,:,2.30,354 x 259 x 22.6,AMD Ryzen 9 Octa Core - 5900HX,3.3 GHz,8,NVIDIA GeForce RTX 3080,16 MB,SSD,2 TB,4-cell Li-ion,240W
5,ASUS,-,Windows 10 Home,1920 x 1080,15.6,-,16 GB,:,2.30,35.4 x 25.9 x 2.26,AMD Ryzen™ 9 5900HX,3.3 GHz,-,NVIDIA® GeForce RTX™ 3070,16M,SSD,1 TB,Li-ion,240W
6,ROG Zephyrus G14,Gaming,Windows 10 Home,1920 x 1080,14,Yes,16 GB,:,1.65,32.5 x 22.1 x 1.8,AMD 3rd Gen Ryzen 9,3.3 GHz,Quad Core,NVIDIA GeForce RTX 2060,8 MB,SSD,1 TB,Lithium,180W
7,Legion 5i,-,Windows 10 Pro,1920 x 1080,15.6,-,16 GB,:,2.3,363.06 x 259.61 x 23.57,10th Gen Intel® Core™ i5-10300H,2.50 GHz,4,NVIDIA® GeForce® GTX 1650 4GB,8 MB,SSD,1 TB,-,-
8,ROG Zephyrus Duo 15,-,Windows 10,3840 x 1100,15.6,-,4 GB,:,2.4,268.30 x 360.00 x 20.90,Intel Core i7 10th Gen 10875H,-,-,NVIDIA GeForce RTX 2070 Max-Q,-,SSD,512 GB,-,-
9,Aspire 7 Gaming,-,Windows 10 Home,1920 x 1080,15.6,5.1,8 GB,:,2.15,2.29 x 36.3 x 25.4,AMD Ryzen™ 5-5500U hexa-core,-,-,NVIDIA® GeForce® GTX 1650,-,SSD,512 GB,Li-ion,135 W


# 8. 
Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be 
scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.


In [50]:
def billionaires():
    driver=webdriver.Chrome('Chromedriver.exe')
    driver.get('https://www.forbes.com/') #loading url
    driver.maximize_window()         
    #CLicking Menu button
    driver.find_element_by_xpath('/html/body/div[1]/header/nav/div[1]/button[1]').click()

    #Hovering over to Billinaires, then to world billionaires and the clicking
    action=ActionChains(driver)
    billionaires=driver.find_element_by_xpath('//div[1]/header/nav/div[3]/ul/li[1]')
    world=driver.find_element_by_xpath('/html/body/div[1]/header/nav/div[3]/ul/li[1]/div[2]/ul/li[2]/a')
    action.move_to_element(billionaires).move_to_element(world).click().perform()
    time.sleep(3)

    #Initializing lists to store the features
    rank=[]
    name=[]
    worth=[]
    age=[]
    country=[]
    source=[]
    industry=[]

    while True:

        #Extracting each feature
        for i in driver.find_elements_by_xpath('//div[@class="rank"]'):
            rank.append(i.text)
        for i in driver.find_elements_by_xpath('//div[@class="personName"]/div'):
            name.append(i.text)
        for i in driver.find_elements_by_xpath('//div[@class="netWorth"]/div'):
            worth.append(i.text)
        for i in driver.find_elements_by_xpath('//div[@class="age"]/div'):
            age.append(i.text)
        for i in driver.find_elements_by_xpath('//div[@class="countryOfCitizenship"]'):
            country.append(i.text)
        for i in driver.find_elements_by_xpath('//div[@class="source-column"]/div[1]'):
            source.append(i.text)
        for i in driver.find_elements_by_xpath('//div[@class="category"]/div'):
            industry.append(i.text)

        try:
            #Waiting for the next button to appear then clicking on it
            delay=10
            WebDriverWait(driver,delay).until(EC.presence_of_element_located((By.XPATH,'//*[@id="gatsby-focus-wrapper"]/div/div/div[3]/div[2]/div[2]/div[2]/div[27]/div[7]/div[2]/button[2]')))
            driver.find_element_by_xpath('//*[@id="gatsby-focus-wrapper"]/div/div/div[3]/div[2]/div[2]/div[2]/div[27]/div[7]/div[2]/button[2]').click()
            time.sleep(2)
        except TimeoutException:
            break #If next button is not found that means we are at the end, hence we come out of the loop
        

    #Creating DataFrame
    data=pd.DataFrame()
    data['Rank']=rank
    data['Name']=name
    data['Worth']=worth
    data['Age']=age
    data['Citizenship']=country
    data['Source']=source
    data['Industry']=industry
    return data
    

In [51]:
df=billionaires()
df 

<ipython-input-50-df4f536924ad>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome('Chromedriver.exe')
<ipython-input-50-df4f536924ad>:6: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('/html/body/div[1]/header/nav/div[1]/button[1]').click()
<ipython-input-50-df4f536924ad>:10: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  billionaires=driver.find_element_by_xpath('//div[1]/header/nav/div[3]/ul/li[1]')
<ipython-input-50-df4f536924ad>:11: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  world=driver.find_element_by_xpath('/html/body/div[1]/header/nav/div[3]/ul/li[1]/div[2]/ul/li[2]/a')
<ipython-input-50-df4f536924ad>:27: DeprecationWarning: find_elements_by_xpath is deprecated. Please use

,Rank,Name,Worth,Age,Citizenship,Source,Industry
0,1.,Jeff Bezos,$177 B,57,United States,Amazon,Technology
1,2.,Elon Musk,$151 B,49,United States,"Tesla, SpaceX",Automotive
2,3.,Bernard Arnault & family,$150 B,72,France,LVMH,Fashion & Retail
3,4.,Bill Gates,$124 B,65,United States,Microsoft,Technology
4,5.,Mark Zuckerberg,$97 B,36,United States,Facebook,Technology
...,...,...,...,...,...,...,...
2750,2674.,Daniel Yong Zhang,$1 B,49,China,e-commerce,Technology
2751,2674.,Zhang Yuqiang,$1 B,65,China,Fiberglass,Manufacturing
2752,2674.,Zhao Meiguang,$1 B,58,China,gold mining,Metals & Mining
2753,2674.,Zhong Naixiong,$1 B,58,China,conglomerate,Diversified


# 9. 
Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted 
from any YouTube Video

In [77]:
def utube_comments(no_of_comments):
    driver=webdriver.Chrome('Chromedriver.exe')
    driver.get('https://www.youtube.com/')    #loading url
    driver.maximize_window()                  #maximizing window

    search=driver.find_element_by_xpath('//input[@id="search"]')
    search.send_keys('all time low')
    search.send_keys(Keys.ENTER)
    time.sleep(2)

    WebDriverWait(driver,10).until(EC.presence_of_element_located((By.ID,'video-title')))
    driver.find_element_by_xpath('//a[@id="video-title"]').click()

    comment=[]
    upvote=[]
    tim=[]

    f=0
    while f==0:
        try:
            flag=driver.find_element_by_xpath('//ytd-comment-thread-renderer['+str(no_of_comments)+']/ytd-comment-renderer/div[2]/div[2]/ytd-expander/div/yt-formatted-string[2]')
            f=1
            time.sleep(3)
        except NoSuchElementException:
            driver.execute_script("window.scrollBy(0,500)")

    for i in driver.find_elements_by_xpath('//*[@id="content-text"]'):
        comment.append(i.text)
    for i in driver.find_elements_by_xpath('//*[@id="vote-count-middle"]'):
        upvote.append(i.text)
    for i in driver.find_elements_by_xpath('//div[@id="header-author"]/yt-formatted-string/a'):
        tim.append(i.text)
    
    data1=pd.DataFrame()
    data1['Comment']=comment
    data1['Upvote']=upvote
    data1['Time when comment was posted']=tim
    return data

In [78]:
no_of_comments=500
data1=utube_comments(no_of_comments)
data1

<ipython-input-77-59fd76275235>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome('Chromedriver.exe')
<ipython-input-77-59fd76275235>:6: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  search=driver.find_element_by_xpath('//input[@id="search"]')
<ipython-input-77-59fd76275235>:12: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//a[@id="video-title"]').click()


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//a[@id="video-title"]"}
  (Session info: chrome=99.0.4844.82)
Stacktrace:
Backtrace:
	Ordinal0 [0x00369943+2595139]
	Ordinal0 [0x002FC9F1+2148849]
	Ordinal0 [0x001F4528+1066280]
	Ordinal0 [0x00220FD4+1249236]
	Ordinal0 [0x002211CB+1249739]
	Ordinal0 [0x0024D812+1431570]
	Ordinal0 [0x0023BA34+1358388]
	Ordinal0 [0x0024BAF2+1424114]
	Ordinal0 [0x0023B806+1357830]
	Ordinal0 [0x00216086+1204358]
	Ordinal0 [0x00216F96+1208214]
	GetHandleVerifier [0x0050B232+1658114]
	GetHandleVerifier [0x005C312C+2411516]
	GetHandleVerifier [0x003FF261+560433]
	GetHandleVerifier [0x003FE366+556598]
	Ordinal0 [0x0030286B+2173035]
	Ordinal0 [0x003075F8+2192888]
	Ordinal0 [0x003076E5+2193125]
	Ordinal0 [0x003111FC+2232828]
	BaseThreadInitThunk [0x77096739+25]
	RtlGetFullPathName_UEx [0x772B8E7F+1215]
	RtlGetFullPathName_UEx [0x772B8E4D+1165]
	(No symbol) [0x00000000]


# 10.
Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in 
“London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, 
overall reviews, privates from price, dorms from price, facilities and property description

In [102]:
# Activating the chrome browser
driver=webdriver.Chrome("chromedriver.exe") 
time.sleep(3)

# Opening the homepage of hostelworld.com
url = "https://www.hostelworld.com/"
driver.get(url)

#locating the location search bar
loc = driver.find_element_by_xpath('/html/body/div[3]/div/div/div[1]/div[2]/div/div[2]/div[4]/div/div[2]/div/div[1]/div/div/div/input')
# write Lonodn in search bar
loc.send_keys("London")
time.sleep(2)

#select london
london = driver.find_element_by_xpath('/html/body/div[1]/div/div/div[1]/div[1]/div/div[2]/div[4]/div/div[2]/div/div[1]/div/div/ul/li[2]/div')
london.click()
time.sleep(2)


# do click in search button
search_btn = driver.find_element_by_id('search-button')
search_btn.click()

# Make empty lists
Hostel_Name = []
Distance = []
overall_review = []
total_reviews = []
facilities = []
price = []
Rating = []
property_description = []

while(True):
    
    # Hostel name
    names = driver.find_elements_by_xpath('//h2[@class="title title-6"]')
    for name in names:
        Hostel_Name.append(name.text)
    time.sleep(2)
        
    # Distance from city centre
    dis = driver.find_elements_by_xpath('//span[@class="description"]')
    for d in dis:
        Distance.append(d.text)
    time.sleep(2)
        
    # Overall Review    
    review = driver.find_elements_by_xpath('//div[@class="keyword"]//span')
    for r in review:
        overall_review.append(r.text)
    time.sleep(2)
    
    # Total No of reviews     
    t_review = driver.find_elements_by_xpath('//div[@class="reviews"]')
    for tr in t_review:
        total_reviews.append(tr.text)
    time.sleep(2)
    
    # facilities
    service = driver.find_elements_by_xpath('//div[@class="facilities-label facilities"]')
    for s in service:
        facilities.append(s.text)
    time.sleep(2)
    
    # Prices    
    prices = driver.find_elements_by_xpath('//div[@class="price-col"]')
    for p in prices:
        price.append(p.text)
    time.sleep(2)    
        
    try:
        next_button = driver.find_element_by_xpath('//div[@class="pagination-item pagination-next"]')
        next_button.click()
    except:
        break
        
time.sleep(4)        
        
# Separate  Privates_From price  and  Dorms_From price
Privates_From = []
for i in range(0,len(price),2):
    Privates_From.append(price[i])
time.sleep(2)

Dorms_From = []
for i in range(1,len(price),2):
    Dorms_From.append(price[i])           
    
    # scrape Hostels URL
hostel_url = []

while(True):
    urls = driver.find_elements_by_xpath('//h2[@class="title title-6"]//a')
    for url in urls:
        hostel_url.append(url.get_attribute("href"))
    time.sleep(2)    
        
    try:
        next_button = driver.find_element_by_xpath('//div[@class="pagination-item pagination-next"]')
        next_button.click()
    except:
        break


        
Rate = []
for page in hostel_url:
    driver.get(page)
    
    # Rating
    try:
        ratings = driver.find_element_by_xpath('/html/body/div[1]/div/div/div[1]/section/div[6]/div/div[1]/div[1]/div[1]')
        Rate.append(ratings.text)
    except NoSuchElementException:
        Rate.append("No Rating")  
    time.sleep(2)
    
    
    # Property Description
    try:
        pd = driver.find_element_by_xpath('/html/body/div[1]/div/div/div[1]/section/div[6]/div/div[2]/div[2]/div/div[2]')
        property_description.append(pd.text)
    except NoSuchElementException:
        property_description.append("No Description")  

    
time.sleep(2)        
# remove extra data from Rating     
all_text = []
for i in Rate:
    all_text.append(i.split())
time.sleep(2)

for i in all_text:
    Rating.append(i[0]) 
    
#creating dataframe
df = pd.DataFrame({'Hostel Name':Hostel_Name,
                'Distance from city centre':Distance,
                'Overall Review':overall_review[3:],
                'Total Reviews':total_reviews,
                'Facilities':facilities,
                'Privates From Price':Privates_From,
                'Dorms From Price':Dorms_From,
                'Rating':Rating,
                'Property Description':property_description})
df

<ipython-input-102-7e2b2bc83151>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome("chromedriver.exe")
<ipython-input-102-7e2b2bc83151>:10: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  loc = driver.find_element_by_xpath('/html/body/div[3]/div/div/div[1]/div[2]/div/div[2]/div[4]/div/div[2]/div/div[1]/div/div/div/input')


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div/div/div[1]/div[2]/div/div[2]/div[4]/div/div[2]/div/div[1]/div/div/div/input"}
  (Session info: chrome=99.0.4844.82)
Stacktrace:
Backtrace:
	Ordinal0 [0x00369943+2595139]
	Ordinal0 [0x002FC9F1+2148849]
	Ordinal0 [0x001F4528+1066280]
	Ordinal0 [0x00220FD4+1249236]
	Ordinal0 [0x002211CB+1249739]
	Ordinal0 [0x0024D812+1431570]
	Ordinal0 [0x0023BA34+1358388]
	Ordinal0 [0x0024BAF2+1424114]
	Ordinal0 [0x0023B806+1357830]
	Ordinal0 [0x00216086+1204358]
	Ordinal0 [0x00216F96+1208214]
	GetHandleVerifier [0x0050B232+1658114]
	GetHandleVerifier [0x005C312C+2411516]
	GetHandleVerifier [0x003FF261+560433]
	GetHandleVerifier [0x003FE366+556598]
	Ordinal0 [0x0030286B+2173035]
	Ordinal0 [0x003075F8+2192888]
	Ordinal0 [0x003076E5+2193125]
	Ordinal0 [0x003111FC+2232828]
	BaseThreadInitThunk [0x77096739+25]
	RtlGetFullPathName_UEx [0x772B8E7F+1215]
	RtlGetFullPathName_UEx [0x772B